In [1]:
import torch

from torch.utils.data import DataLoader

from utils.load import MetaLoader, DialogueTrainLoader
from utils.preprocess import Preprocessor, Augmentation
from utils.encoder import Encoder

from data import FH2024Dataset, collate_fn
from net.tokenizer import SubWordEmbReaderUtil
from net.model import Model

In [2]:
swer_path = './sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat'
swer = SubWordEmbReaderUtil(swer_path)

meta_path = '../../datasets/FH_2024/subtask3/mdata.wst.txt.2023.08.23'
meta_loader = MetaLoader(path=meta_path, swer=swer)
img2id, id2img, img_similarity = meta_loader.get_dataset()


<Initialize subword embedding>
loading= ./sstm_v0p5_deploy/sstm_v4p49_np_n36134_d128.dat


In [3]:
train_path = "../../datasets/FH_2024/subtask3/task1.ddata.wst.txt"
train_diag_loader = DialogueTrainLoader(path=train_path)
train_raw_dataset = train_diag_loader.get_dataset()

In [4]:
preprocessor = Preprocessor(num_rank=3, num_coordi=4, threshold=0.7)
train_dataset = preprocessor(train_raw_dataset, img2id, id2img, img_similarity)

In [5]:
augmentation = Augmentation(num_aug=5, num_rank=3, num_coordi=4, threshold=0.7)
train_dataset = augmentation(train_dataset, img2id, id2img, img_similarity)

In [6]:
encoder = Encoder(swer=swer, img2id=img2id, num_coordi=4, mem_size=16, meta_size=4)
encoded_train_dataset = encoder(train_dataset)

In [7]:
# -- Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -- Training
BATCH_SIZE = 16
EPOCH = 10

# -- Model
KEY_SIZE = 300
MEM_SIZE = 16
HOPS = 3
EVAL_NODE = '[6000,6000,6000,200][2000,2000]'
DROP_PROB = 0.1

In [8]:
train_dataset = FH2024Dataset(dataset=encoded_train_dataset)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, collate_fn=collate_fn)

In [9]:
item_size = [len(img2id[i]) for i in range(4)] # 각 카테고리별 개수
net = Model(emb_size=swer.get_emb_size(), 
            key_size=KEY_SIZE, 
            mem_size=MEM_SIZE,
            meta_size=4, 
            hops=HOPS, 
            item_size=item_size, 
            coordi_size=4,
            eval_node=EVAL_NODE, 
            num_rnk=3, 
            use_batch_norm=False, 
            use_dropout=True,
            zero_prob=DROP_PROB,
            use_multimodal=False,
            img_feat_size=4096)

In [10]:
# 필요한 추가 임포트
import torch

# train_loader에서 하나의 배치만 가져오기
for batch in train_loader:
    break

# GPU 사용 가능시 모델을 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = net.to(device)

# 배치의 첫 번째 샘플 선택
desc = batch['description'][0].unsqueeze(0).to(device)  # shape: (1, mem_size, emb_size)
coordi = batch['coordi'][0].unsqueeze(0).to(device)  # shape: (1, num_rank, emb_size * coordi_size)
actual_rank = batch['rank'][0].item()

# 모델 실행
net.eval()  # 평가 모드 설정
with torch.no_grad():
    logits = net(desc, coordi)
    preds = torch.argsort(logits, -1, descending=True).cpu().numpy()[0]

print(logits.shape)

/home/ho/workspace/FH_2024/subtask3/data.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coordi_tensor = torch.tensor(coordi_tensor, dtype=torch.long)
/home/ho/workspace/FH_2024/subtask3/data.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  coordi_tensor = torch.tensor(coordi_tensor, dtype=torch.long)
/home/ho/workspace/FH_2024/subtask3/data.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internall

torch.Size([1, 6])


In [ ]:
from scipy import stats

In [3]:
def _calculate_weighted_kendall_tau(pred, label):
    total_count = 0
    total_corr = 0.0

    for p, l in zip(pred, label):
        corr, _ = stats.weightedtau(p, l)
        total_corr += corr
        total_count += 1
    
    return (total_corr / total_count)

In [4]:
label = [[0, 1, 2], [0, 2, 1], [1, 0, 2], [1, 2, 0]]
pred = [[1, 0, 2], [1, 2, 0], [0, 1, 2], [0, 2, 1]]

corr = _calculate_weighted_kendall_tau(pred, label)
print(f"WKT: {corr: .4f}")

WKT:  0.5455
